In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import torchvision.models as models
import cv2


In [3]:
class Extractor(nn.Module):
    def __init__(self):
        super(Extractor, self).__init__()
        
        self.net = models.densenet121(pretrained=True).features         
        
    def forward(self, images): 
        return self.net(images)

In [4]:
def read_img(img_path):
    img = plt.imread(img_path)
    img = cv2.resize(img,(224,224))
    img = img / 255
    img = img.transpose((2,0,1))
    return img
    

In [5]:
with open("./food-101/meta/train.txt","r") as f:
    training_list = f.readlines()
with open("./food-101/meta/test.txt","r") as f:
    test_list = f.readlines()
    
with open('./food-101/meta/labels.txt','r') as f:
    labels = f.readlines()
    
training_list = ["./food-101/images/"+ path[:-1] +".jpg" for path in training_list]
test_list = ["./food-101/images/"+ path[:-1] +".jpg" for path in test_list]
labels = [label[:-1].replace(" ",'_').lower() for label in labels]

In [6]:
net = Extractor()
net.cuda()

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Extractor(
  (net): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
 

In [11]:
import pickle

test_data = []
counter = 0
for i in range(len(test_list)):
    img = read_img(test_list[i])
    x = torch.FloatTensor(img).view(1,3,224,224)
    x = Variable(x.cuda())
    feature = net(x)
    label_id = labels.index(test_list[i].split("/")[-2])
    test_data.append((label_id,feature.cpu().data.numpy()))
    if i % 1000 == 0:
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000


In [ ]:
 with open("test_data_densenet.pkl","wb") as f:
        pickle.dump(test_data, f)

In [13]:
i

25249

In [ ]:
import pickle

training_data = []
counter = 0
for i in range(len(training_list)):
    img = read_img(training_list[i])
    x = torch.FloatTensor(img).view(1,3,224,224)
    x = Variable(x.cuda())
    feature = net(x)
    label_id = labels.index(training_list[i].split("/")[-2])
    training_data.append((label_id,feature.cpu().data.numpy()))
    if i % 10000 == 0:
        print(i)

0
10000
20000
30000


In [19]:
len(training_data)

75747

In [20]:
 with open("training_data.pkl","wb") as f:
        pickle.dump(training_data, f)

In [29]:
training_data[-1]

(100, array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32))